In [41]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://archive.ubuntu.com/ubun

In [42]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-06 23:24:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-04-06 23:24:23 (1.69 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [43]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Vine-Challenge").getOrCreate()

In [44]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [45]:
# Create the vine DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

In [46]:
# Filter the dataframe to get only reviews with 20 or more votes
filtered_df = vine_df.filter("total_votes >= 20")
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R26UFXLLQ2N0CR|          1|            0|         20|   N|                Y|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|     

In [47]:
# Find only reviews where more than 50 percent of the votes were for helpful
helpful_df = filtered_df.filter("helpful_votes / total_votes >= 0.5")
helpful_df = helpful_df.withColumn("vote_percentage", helpful_df.helpful_votes / helpful_df.total_votes)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   vote_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|0.9124087591240876|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|0.9333333333333333|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|0.8846153846153846|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|0.7619047619047619|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|0.8947368421052632|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|             0.875|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|0.8846153846153846|
|R2M6WNM2OZZHKS|          1|  

In [48]:
# Create dataframes for reviews that were a vine and those that were not
vine_true_df = helpful_df.filter("vine = 'Y'")
vine_false_df = helpful_df.filter("vine = 'N'")

vine_true_df.show(10)
vine_false_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   vote_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N| 0.918918918918919|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|0.8648648648648649|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|              0.88|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|          0.953125|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|               1.0|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|0.9655172413793104|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|0.9806201550387597|
|R13EGJI8C8YJ4Y|          5|  

In [49]:
# Determine the total number of star ratings for each dataframe

# Function for summarizing ratings
def rating_summary(df):
  summary_df = df.groupBy("star_rating").count().sort("star_rating")
  total_ratings = summary_df.agg({'count':'sum'}).collect()[0]["sum(count)"]
  summary_df = summary_df.withColumn('rating_percent', summary_df['count'] / total_ratings * 100)
  print(total_ratings)
  return summary_df, total_ratings

vine_true_summary, vine_true_total = rating_summary(vine_true_df)
vine_false_summary, vine_false_total = rating_summary(vine_false_df)

61
28287


In [50]:
vine_true_summary.show()
vine_false_summary.show()

+-----------+-----+------------------+
|star_rating|count|    rating_percent|
+-----------+-----+------------------+
|          1|    7|11.475409836065573|
|          2|    6| 9.836065573770492|
|          3|   11|  18.0327868852459|
|          4|   17| 27.86885245901639|
|          5|   20| 32.78688524590164|
+-----------+-----+------------------+

+-----------+-----+------------------+
|star_rating|count|    rating_percent|
+-----------+-----+------------------+
|          1| 6412|  22.6676565206632|
|          2| 1655| 5.850744158093824|
|          3| 1931| 6.826457383250256|
|          4| 2600| 9.191501396401174|
|          5|15689|55.463640541591545|
+-----------+-----+------------------+

